In [1]:
import time

import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.nn import VGAE
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree, negative_sampling

from torch.utils.tensorboard import SummaryWriter

from gene_graph_dataset import G3MedianDataset
from phylognn_model import G3Median_GCNConv

In [2]:
gpuid = 0

train_p, test_p = 0.8, 0.2
train_batch, test_batch = 512, 64

In [3]:
device = torch.device('cuda:' + str(gpuid) if torch.cuda.is_available() else 'cpu')

In [4]:
dataset = G3MedianDataset('dataset_g3m', 100, 100)

In [5]:
data_size = len(dataset)
train_size, test_size = (int)(data_size * train_p), (int)(data_size * test_p)

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:(train_size + test_size)]
# val_dataset = dataset[(train_size + test_size):]

In [7]:
test_dataset = list(test_dataset)
for t in test_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)

In [8]:
train_loader = DataLoader(train_dataset, batch_size = train_batch, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size = test_batch)
# val_loader = DataLoader(val_dataset, batch_size=8)

In [9]:
# deg = torch.zeros(5, dtype=torch.long)
# for data in train_dataset:
#     d = degree(data.edge_index[1].type(torch.int64), 
#                num_nodes=data.num_nodes, dtype=torch.long)
#     deg += torch.bincount(d, minlength=deg.numel())

In [10]:
in_channels, out_channels = dataset.num_features, 16

In [11]:
# class G3M_VGAE(VGAE):
#     def __init__(self, encoder, decoder = None):
#         super(G3M_VGAE, encoder, decoder)
        
#     def recon_loss(self, z, pos_edge_label_index):
#         pred = self.decoder.forward_all(z)
        
        

In [12]:
model = VGAE(G3Median_GCNConv(in_channels, out_channels)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10,
                              min_lr=0.00001)

In [13]:
writer = SummaryWriter(log_dir='runs_g3m/g3median_lbatch_gcn_aneg')

In [14]:
def train(train_loader):
    model.train()
    
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.pos_edge_label_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    return total_loss/len(train_loader)

In [15]:
@torch.no_grad()
def test(test_loader):
    model.eval()
    loss, auc, ap = 0, 0, 0
    
    for data in test_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        loss += model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
                                 # negative_sampling(data.pos_edge_label_index, 
                                                # data.num_nodes,
                                                # data.num_nodes*10))
                                 # data.neg_edge_label_index)
        tauc, tap = model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)
                               # negative_sampling(data.pos_edge_label_index, 
                                                # data.num_nodes,
                                                # data.num_nodes*10))
                               # data.neg_edge_label_index)
        auc += tauc
        ap += tap
        
    return loss/len(test_loader), auc/len(test_loader), ap/len(test_loader)

In [16]:
for epoch in range(1, 200 + 1):
    print(f'{time.ctime()} - Epoch: {epoch:04d}')
    loss = train(train_loader)
    print(f'{time.ctime()} - \t train loss: {loss:.6f}')
    tloss, auc, ap = test(test_dataset)
    print(f'{time.ctime()} - \t test  loss: {tloss:.6f}, auc: {auc:.6f}, ap: {ap:.6f}')
    scheduler.step(1 - auc)
    
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Loss/test', tloss, epoch)
    writer.add_scalar('AUC/test', auc, epoch)
    writer.add_scalar('AP/test', ap, epoch)
    # if epoch % 50 == 0:
    # print(f'{time.ctime()} - '
          # f'Epoch: {epoch:04d}, loss: {loss:.6f}, AUC: {auc:.6f}, '
          # f', TL: {tloss:.6f}')

Sun Dec 26 16:24:06 2021 - Epoch: 0001
Sun Dec 26 16:24:17 2021 - 	 train loss: 3.597025
Sun Dec 26 16:25:39 2021 - 	 test  loss: 1.334841, auc: 0.841267, ap: 0.033663
Sun Dec 26 16:25:39 2021 - Epoch: 0002
Sun Dec 26 16:25:48 2021 - 	 train loss: 0.980776
Sun Dec 26 16:27:07 2021 - 	 test  loss: 1.020093, auc: 0.939781, ap: 0.119717
Sun Dec 26 16:27:07 2021 - Epoch: 0003
Sun Dec 26 16:27:16 2021 - 	 train loss: 0.929141
Sun Dec 26 16:28:34 2021 - 	 test  loss: 0.947832, auc: 0.957284, ap: 0.156371
Sun Dec 26 16:28:34 2021 - Epoch: 0004
Sun Dec 26 16:28:43 2021 - 	 train loss: 0.909711
Sun Dec 26 16:30:02 2021 - 	 test  loss: 0.926073, auc: 0.963109, ap: 0.178171
Sun Dec 26 16:30:02 2021 - Epoch: 0005
Sun Dec 26 16:30:11 2021 - 	 train loss: 0.900544
Sun Dec 26 16:31:31 2021 - 	 test  loss: 0.918335, auc: 0.965959, ap: 0.190617
Sun Dec 26 16:31:31 2021 - Epoch: 0006
Sun Dec 26 16:31:40 2021 - 	 train loss: 0.893500
Sun Dec 26 16:32:59 2021 - 	 test  loss: 0.913474, auc: 0.967936, ap: 0

In [17]:
writer.close()

In [18]:
# torch.save(model.state_dict(), 'g2g_test_model_batch')